In [ ]:
#in CMD: pip install mysqlclient
import configparser
import os
import sys
import MySQLdb
#add crawler package as additional source of modules
module_path = os.path.abspath(os.path.join('crawler'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from DatabaseHandler import DatabaseHandler 
from model import NewsProvider, NewsArticle

import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np


In [ ]:
conf = configparser.ConfigParser()
conf.read("crawler_config.ini")
dbh = DatabaseHandler(conf['DATABASE']['Host'], conf['DATABASE']['User'], conf['DATABASE']['Password'], conf['DATABASE']['DB'])

## Work with DB Handler and Objects/Dicts

In [ ]:
#provider = dbh.readNewsProvider()
articles = dbh.select("NewsArticles")
articles[0]["source_uri"]

## Plain SQL


In [ ]:
## copied fom DB Handler
def executeSQL(dbh,statement):
    if statement:
        try:
            print('Executing SQL-query:\n\t%s'
                              % statement.replace('\n', '\n\t'))
            dbh.cnx.execute(statement)
            return dbh.cnx.fetchall()
        except MySQLdb.Warning as e:
            print("Warning while executing statement: %s" % e)
        except MySQLdb.Error as e:
            print("Error while executing statement [%d]: %s"
                              % (e.args[0], e.args[1]))

In [ ]:
executeSQL(dbh, """SELECT news_provider, COUNT(*) 
           FROM NewsArticles 
           GROUP BY news_provider 
           ORDER BY COUNT(*) DESC""")

In [ ]:
#Structure of NewsArticles: 'added_on': datetime.datetime(2017, 4, 21, 17, 0, 43),'author': '[]','news_provider': '...', 'source_uri': '...','text'
dates = executeSQL(dbh, """SELECT DATE(added_on) as "date" , COUNT(*) as "count"
           FROM NewsArticles 
           GROUP BY DATE(added_on) 
           ORDER BY DATE(added_on) ASC """)

In [ ]:
executeSQL(dbh, """SELECT *
           FROM NewsArticlesBOW
           LIMIT 1""")

In [ ]:
executeSQL(dbh, """SELECT MAX(LENGTH(text))
           FROM NewsArticles 
           """)

In [ ]:
res = executeSQL(dbh,"""SELECT source_uri, text 
    FROM NewsArticles LIMIT 2    
    """)

In [ ]:
n = len(res)
uris = np.empty(n,dtype=np.dtype(('U', 255)))
texts = np.empty(n,dtype=np.dtype(('U', 10000)))

for i, row in enumerate(res):
    uris[i] = row["source_uri"]
    texts[i] = row["text"]

In [ ]:
texts

In [ ]:
counts = []
values = []
for date in dates: 
    counts.append(date["count"])
    #values.append(str(date["date"].day) + "." + str(date["date"].month))
    values.append(date["date"].day)


In [ ]:
plt.plot(values,counts) 
plt.axis([min(values),max(values),0, max(counts)])

In [ ]:
values